to try to implement what I learned in that uproot/awkward tutorial and create some 4-vectors of pt, eta, phi, and mass (starting with the delta dataset) using awkward and vector. Then to try to get some deltaR plots working!

NOTE: this one was with the old versions, I'm just gonna leave it like this for posterity and research purposes (read: too lazy to clean it up) and make a nice, neat, new one:))

In [1]:
#imports
import uproot as ur
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import vector as v

data_path = '/fast_scratch/atlas_images/v01-45/'

In [51]:
v.register_awkward()

In [23]:
branches = ['cluster_hitsTruthIndex', 'cluster_hitsTruthE', 'truthPartPdgId', 'truthPartPt','truthPartEta', 
            'truthPartPhi', 'truthPartMass']

In [4]:
event = ur.open(data_path+'pi0_small.root')

In [10]:
event.keys()

['EventTree;1', 'CellGeo;1']

In [11]:
tree = event['EventTree']

In [16]:
tree.keys()

['runNumber',
 'eventNumber',
 'lumiBlock',
 'coreFlags',
 'mcEventNumber',
 'mcChannelNumber',
 'mcEventWeight',
 'nTruthPart',
 'G4PreCalo_n_EM',
 'G4PreCalo_E_EM',
 'G4PreCalo_n_Had',
 'G4PreCalo_E_Had',
 'truthVertexX',
 'truthVertexY',
 'truthVertexZ',
 'truthPartPdgId',
 'truthPartStatus',
 'truthPartBarcode',
 'truthPartPt',
 'truthPartE',
 'truthPartMass',
 'truthPartEta',
 'truthPartPhi',
 'nTrack',
 'trackPt',
 'trackP',
 'trackMass',
 'trackEta',
 'trackPhi',
 'trackNumberOfPixelHits',
 'trackNumberOfSCTHits',
 'trackNumberOfPixelDeadSensors',
 'trackNumberOfSCTDeadSensors',
 'trackNumberOfPixelSharedHits',
 'trackNumberOfSCTSharedHits',
 'trackNumberOfPixelHoles',
 'trackNumberOfSCTHoles',
 'trackNumberOfInnermostPixelLayerHits',
 'trackNumberOfNextToInnermostPixelLayerHits',
 'trackExpectInnermostPixelLayerHit',
 'trackExpectNextToInnermostPixelLayerHit',
 'trackNumberOfTRTHits',
 'trackNumberOfTRTOutliers',
 'trackChiSquared',
 'trackNumberDOF',
 'trackD0',
 'trackZ0',
 '

In [21]:
tree.arrays(library='np')

{'runNumber': array([284500, 284500, 284500, ..., 284500, 284500, 284500], dtype=int32),
 'eventNumber': array([1453941, 1453785, 1453509, ..., 2933860, 2933060, 2933409]),
 'lumiBlock': array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 'coreFlags': array([0, 0, 0, ..., 0, 0, 0], dtype=uint32),
 'mcEventNumber': array([1453941, 1453785, 1453509, ..., 2933860, 2933060, 2933409],
       dtype=int32),
 'mcChannelNumber': array([900246, 900246, 900246, ..., 900246, 900246, 900246], dtype=int32),
 'mcEventWeight': array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
 'nTruthPart': array([3, 3, 3, ..., 3, 3, 3], dtype=int32),
 'G4PreCalo_n_EM': array([ 0, 37,  0, ...,  4,  0,  0], dtype=int32),
 'G4PreCalo_E_EM': array([   0.      , 1970.3767  ,    0.      , ...,   16.828182,
           0.      ,    0.      ], dtype=float32),
 'G4PreCalo_n_Had': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'G4PreCalo_E_Had': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'truthVertexX': array([-0.5014897

In [24]:
branch = tree.arrays(expressions=branches)

In [26]:
branch.fields #aha now I can do fields!

['cluster_hitsTruthIndex',
 'cluster_hitsTruthE',
 'truthPartPdgId',
 'truthPartPt',
 'truthPartEta',
 'truthPartPhi',
 'truthPartMass']

In [27]:
branch[0].tolist()

{'cluster_hitsTruthIndex': [[2, 1], [2, 1]],
 'cluster_hitsTruthE': [[674.6878662109375, 209.26992797851562],
  [7.056183815002441, 2.905189037322998]],
 'truthPartPdgId': [111, 22, 22],
 'truthPartPt': [861.3355712890625, 204.65724182128906, 656.6782836914062],
 'truthPartEta': [0.32378026843070984,
  0.3237888216972351,
  0.32377761602401733],
 'truthPartPhi': [0.06045312061905861,
  0.06017250195145607,
  0.0605405755341053],
 'truthPartMass': [0.13500000536441803, 0.0, -1.3531646800402086e-05]}

In [28]:
branch.truthPartPhi #okay so this works, but it is not in the same format as the other thing...

<Array [[0.0605, 0.0602, ... -1.18, -1.17]] type='200000 * var * float32'>

In [94]:
#maybe try the nanoaod thing??

nanoaod_style = tree.arrays(filter_name="truth*", entry_stop=5)
nanoaod_style.type

5 * {"truthVertexX": float32, "truthVertexY": float32, "truthVertexZ": float32, "truthPartPdgId": var * int32, "truthPartStatus": var * int32, "truthPartBarcode": var * int32, "truthPartPt": var * float32, "truthPartE": var * float32, "truthPartMass": var * float32, "truthPartEta": var * float32, "truthPartPhi": var * float32}

In [95]:
print(nanoaod_style.truthPartEta)

[[0.324, 0.324, 0.324], [1.5, 1.5, 1.5, ... -2.33, -2.33, -2.33], [2.36, 2.37, 2.36]]


In [96]:
nanoevents_style = ak.zip({
    "pt": nanoaod_style.truthPartPt,
    "eta": nanoaod_style.truthPartEta,
    "phi": nanoaod_style.truthPartPhi,
    "mass": nanoaod_style.truthPartMass,
})
nanoevents_style

<Array [[{pt: 861, eta: 0.324, ... mass: 0}]] type='5 * var * {"pt": float32, "e...'>

In [97]:
print(nanoevents_style['pt'])

[[861, 205, 657], [838, 704, 133], ... 0.308], [120, 40.8, 79.5], [177, 100, 76.4]]


In [115]:
print(nanoevents_style['eta'])

[[0.324, 0.324, 0.324], [1.5, 1.5, 1.5, ... -2.33, -2.33, -2.33], [2.36, 2.37, 2.36]]


In [116]:
print(nanoevents_style['mass'])

[[0.135, 0, -1.35e-05], [0.135, 2.21e-05, ... -5.52e-06, 0], [0.135, -1.1e-05, 0]]


In [117]:
print(nanoevents_style['phi'])

[[0.0605, 0.0602, 0.0605], [2.02, 2.02, 2.02, ... -3.01, -3.02], [3.05, 3.05, 3.05]]


In [78]:
ak.unzip(nanoevents_style)

(<Array [[861, 205, 657, ... 4.96, 4.32, 0.643]] type='10 * var * float32'>,
 <Array [[0.324, 0.324, ... 0.696, 0.739]] type='10 * var * float32'>,
 <Array [[0.0605, 0.0602, ... -0.535, -0.467]] type='10 * var * float32'>,
 <Array [[0.135, 0, ... 0, -1.87e-08]] type='10 * var * float32'>)

In [121]:
muonstest = ak.Array([
    [{'pt': 5, 'eta': 1.5, 'phi': 0.1, 'mass': 0.105},
     {'pt': 10.0, 'eta': -1.5, 'phi': 0.2, 'mass': 0.105}],
     {'pt': 5.0, 'eta': 1.5, 'phi': 0.1, 'mass': 0.105},
     {'pt': 10.0, 'eta': -1.5, 'phi': 0.2, 'mass': 0.105}
], with_name='Momentum4D')

In [124]:
muonstest.pt #yep its a versions issue!!!

ValueError: array does not have azimuthal coordinates (x/y/rho/phi): pt, eta, phi, mass

In [102]:
vec4d = ak.with_name(nanoevents_style, "Momentum4D")

In [103]:
vec4d

<MomentumArray4D [[{pt: 861, eta: 0.324, ... mass: 0}]] type='5 * var * Momentum...'>

In [104]:
xtry = v.Array(vec4d)

In [105]:
xtry.fields

['rho', 'phi', 'eta', 'tau']

In [106]:
xtry[1, 0].cross(xtry[1, 1])

IndexError: list index out of range

In [107]:
ytry = ak.Array(nanoevents_style, with_name='Momentum4D')

In [108]:
ytry.fields

['pt', 'eta', 'phi', 'mass']

In [109]:
ytry[1,0].cross(ytry[1,1])

ValueError: array does not have azimuthal coordinates (x/y/rho/phi): pt, eta, phi, mass

In [110]:
vec4d.fields

['pt', 'eta', 'phi', 'mass']

In [111]:
vec4d[1,0], vec4d[0,1]

(<MomentumRecord4D ... eta: 1.5, phi: 2.02, mass: 0.135} type='Momentum4D["pt": f...'>,
 <MomentumRecord4D ... eta: 0.324, phi: 0.0602, mass: 0} type='Momentum4D["pt": f...'>)

In [112]:
vec4d[0].pt

ValueError: array does not have azimuthal coordinates (x/y/rho/phi): pt, eta, phi, mass

In [113]:
ur.__version__, v.__version__, ak.__version__

('4.0.7', '0.8.1', '1.2.2')

In [114]:
vec4d[1, 0].cross(vec4d[1, 1])

ValueError: array does not have azimuthal coordinates (x/y/rho/phi): pt, eta, phi, mass

In [92]:
vec4d

<MomentumArray4D [[{pt: 861, ... mass: -1.87e-08}]] type='10 * var * Momentum4D[...'>